In [12]:
import pandas as pd
from itertools import combinations
import json

class InputParser:
    """
    Parses the input schema, functional dependencies (FDs), multi-valued dependencies (MVDs),
    and data instances directly from user input, with automatic whitespace removal.
    """
    def __init__(self):
        # Initialize empty lists and variables to store schema, dependencies, MVDs, and data instances
        self.schema = []
        self.dependencies = []
        self.mvd = []
        self.data_instances = None

    def get_user_input(self):
        """
        Collects schema, functional dependencies, multi-valued dependencies, and data instances directly from user input.
        """
        # Collect schema attributes as a comma-separated list
        self.schema = [attr.strip() for attr in input("Enter schema attributes (comma-separated): ").split(',')]

        # Collect functional dependencies (FDs) in the format 'A,B -> C'
        print("\nEnter functional dependencies (FDs) in the format 'A,B -> C' (one per line). Type 'done' to finish:")
        while True:
            fd = input()
            if fd.lower() == 'done':
                break
            try:
                left, right = fd.split('->')
                left_attrs = tuple(attr.strip() for attr in left.split(','))
                right_attrs = tuple(attr.strip() for attr in right.split(','))
                self.dependencies.append((left_attrs, right_attrs))
            except ValueError:
                print("Invalid format. Use 'A,B -> C'.")

        # Collect multi-valued dependencies (MVDs) in the format 'A ->-> B'
        print("\nEnter multi-valued dependencies (MVDs) in the format 'A ->-> B' (optional, type 'done' to finish):")
        while True:
            mvd = input()
            if mvd.lower() == 'done':
                break
            try:
                left, right = mvd.split('->->')
                left_attrs = tuple(attr.strip() for attr in left.split(','))
                right_attrs = tuple(attr.strip() for attr in right.split(','))
                self.mvd.append((left_attrs, right_attrs))
            except ValueError:
                print("Invalid format. Use 'A ->-> B'.")

        # Collect data instances as JSON if provided by the user
        data_input = input("\nEnter data instances as JSON (optional). Leave empty if not provided: ")
        if data_input:
            try:
                raw_data_instances = json.loads(data_input)
                # Ensure that all keys in data instances are stripped of extra spaces
                self.data_instances = [{k.strip(): v for k, v in instance.items()} for instance in raw_data_instances]
            except json.JSONDecodeError:
                print("Invalid JSON format for data instances.")

class Normalizer:
    """
    Handles normalization from 1NF to DKNF using schema, FDs, MVDs, and data instances.
    """
    def __init__(self, schema, dependencies, mvd=None, data_instances=None):
        # Initialize schema, dependencies, MVDs, and data instances
        self.schema = schema
        self.dependencies = dependencies
        self.mvd = mvd
        self.data_instances = data_instances
        # List to store the normalized schema after each normalization step
        self.normalized_schema = []
        self.created_tables = set()

    def add_table(self, attributes, primary_key):
        """
        Adds a table to the normalized schema, ensuring no duplicate tables.
        """
        sorted_attrs = tuple(sorted(attributes))
        if sorted_attrs not in self.created_tables:
            self.created_tables.add(sorted_attrs)
            self.normalized_schema.append((attributes, primary_key))

    def is_superkey(self, attrs):
        """
        Checks if a set of attributes forms a superkey.
        """
        closure = set(attrs)
        changed = True
        while changed:
            changed = False
            for left, right in self.dependencies:
                if set(left).issubset(closure) and not set(right).issubset(closure):
                    closure.update(right)
                    changed = True
        return set(self.schema) == closure

    def to_1NF(self):
        """
        Converts schema to 1NF by ensuring all attributes are atomic.
        """
        atomic_schema = [attr for attr in self.schema if ',' not in attr]
        self.schema = atomic_schema
        print("Converted to 1NF: Removed multi-valued attributes.")

    def to_2NF(self):
        """
        Ensures 2NF by removing partial dependencies.
        """
        for left, right in self.dependencies:
            if not self.is_superkey(left) and any(attr in self.schema for attr in right):
                new_table = list(left) + list(right)
                self.add_table(new_table, left)
                self.schema = [attr for attr in self.schema if attr not in right]

    def to_3NF(self):
        """
        Ensures 3NF by removing transitive dependencies.
        """
        for left, right in self.dependencies:
            if not self.is_superkey(left) and set(right).issubset(self.schema):
                new_table = list(left) + list(right)
                self.add_table(new_table, left)
                self.schema = [attr for attr in self.schema if attr not in right]

    def to_BCNF(self):
        """
        Ensures BCNF by ensuring all dependencies have a superkey as the left side.
        """
        for left, right in self.dependencies:
            if not self.is_superkey(left):
                new_table = list(left) + list(right)
                self.add_table(new_table, left)
                self.schema = [attr for attr in self.schema if attr not in right]

    def validate_mvd(self, left, right):
        """
        Validates multi-valued dependency (MVD) with the provided data instances.
        """
        if self.data_instances is None:
            print("Data instances not provided; skipping MVD validation.")
            return True
        df = pd.DataFrame(self.data_instances)
        grouped = df.groupby(list(left))
        for _, group in grouped:
            if len(group[right[0]].unique()) > 1:
                return False
        return True

    def identify_mvds(self):
        """
        Identifies potential MVDs by analyzing data instances.
        """
        if self.data_instances is None:
            print("Data instances not provided; skipping automatic MVD identification.")
            return []
        df = pd.DataFrame(self.data_instances)
        potential_mvds = []
        for left_comb in combinations(self.schema, len(self.schema) - 1):
            for right_attr in set(self.schema) - set(left_comb):
                grouped = df.groupby(list(left_comb))
                if grouped[right_attr].nunique().mean() > 1:
                    potential_mvds.append((left_comb, (right_attr,)))
                    print(f"Automatic MVD detected: {left_comb} ->-> {right_attr}")
        return potential_mvds

    def to_4NF(self):
        """
        Ensures 4NF by handling multi-valued dependencies.
        """
        mvds_to_check = self.mvd or self.identify_mvds()
        for left, right in mvds_to_check:
            if self.validate_mvd(left, right):
                new_table = list(left) + list(right)
                self.add_table(new_table, left)
                self.schema = [attr for attr in self.schema if attr not in right]
            else:
                print(f"MVD {left} ->-> {right} not validated by data instances.")

    def to_5NF(self):
        """
        Ensures 5NF by handling join dependencies.
        """
        if self.data_instances is None:
            print("Data instances not provided; skipping 5NF.")
            return
        df = pd.DataFrame(self.data_instances)
        for attr in self.schema:
            unique_values = df[attr].nunique()
            total_values = len(df[attr])
            if unique_values != total_values:
                new_table = [attr]
                self.add_table(new_table, [attr])
                print(f"Join dependency detected on {attr}, creating new table for 5NF.")

    def to_DKNF(self):
        """
        Ensures Domain-Key Normal Form (DKNF) by enforcing domain constraints.
        """
        if self.data_instances is None:
            print("Data instances not provided; DKNF not applicable without domain constraints.")
            return
        print("Applying DKNF: Ensuring all constraints are domain or key-based.")
        df = pd.DataFrame(self.data_instances)
        for col in self.schema:
            if df[col].dtype == 'object' and df[col].str.isnumeric().any():
                print(f"Domain constraint: Column {col} appears to have numeric values in a non-numeric domain.")
                self.schema.remove(col)
                print(f"Removing {col} to enforce domain constraint for DKNF.")
        self.add_table(self.schema, None)

    def normalize(self, target_nf):
        """
        Applies normalization steps up to the specified normal form.
        """
        self.to_1NF()
        if target_nf >= 2:
            self.to_2NF()
        if target_nf >= 3:
            self.to_3NF()
        if target_nf >= 4:
            self.to_BCNF()
            if self.mvd:
                self.to_4NF()
        if target_nf >= 5:
            self.to_5NF()
        if target_nf > 5:
            self.to_DKNF()
        # Add the final table after all normal forms are applied
        self.add_table(self.schema, None)


class FinalRelationGenerator:
    """
    Generates SQL statements based on the normalized schema.
    """
    def __init__(self, normalized_schema):
        self.normalized_schema = normalized_schema

    def generate_sql(self, output_to_file=False, filename="normalized_schema.sql"):
        """
        Generates SQL CREATE TABLE statements for each normalized relation.
        """
        sql_statements = []
        for i, (table, primary_key) in enumerate(self.normalized_schema):
            table_name = f"Table_{i+1}"
            # Generate CREATE TABLE SQL statement
            sql = f"CREATE TABLE {table_name} (\n"
            sql += ',\n'.join([f"{col} VARCHAR(255)" for col in table])
            if primary_key:
                sql += f",\nPRIMARY KEY ({', '.join(primary_key)})"
            sql += "\n);"
            sql_statements.append(sql)

        # Print each SQL statement
        for sql in sql_statements:
            print(sql)

        # Optionally write SQL statements to a file
        if output_to_file:
            with open(filename, "w") as file:
                file.write("\n\n".join(sql_statements))
            print(f"\nSQL statements saved to {filename}")

        return sql_statements


if __name__ == "__main__":
    # Collect user inputs
    parser = InputParser()
    parser.get_user_input()

    # Get target normalization form level from user
    target_nf = int(input("Enter target normal form (1 to 6): "))

    # Perform normalization
    normalizer = Normalizer(parser.schema, parser.dependencies, parser.mvd, parser.data_instances)
    normalizer.normalize(target_nf)

    # Generate SQL for the normalized schema
    generator = FinalRelationGenerator(normalizer.normalized_schema)
    generator.generate_sql(output_to_file=True)



Enter schema attributes (comma-separated): X, Y, Z, W, V

Enter functional dependencies (FDs) in the format 'A,B -> C' (one per line). Type 'done' to finish:
X -> Y
Y -> Z
Z -> W
done

Enter multi-valued dependencies (MVDs) in the format 'A ->-> B' (optional, type 'done' to finish):
X ->-> V
done

Enter data instances as JSON (optional). Leave empty if not provided: [     {"X": "1", "Y": "2", "Z": "3", "W": "4", "V": "5"},     {"X": "1", "Y": "2", "Z": "3", "W": "4", "V": "6"} ]
Enter target normal form (1 to 6): 5
Converted to 1NF: Removed multi-valued attributes.
MVD ('X',) ->-> ('V',) not validated by data instances.
Join dependency detected on X, creating new table for 5NF.
CREATE TABLE Table_1 (
X VARCHAR(255),
Y VARCHAR(255),
PRIMARY KEY (X)
);
CREATE TABLE Table_2 (
Y VARCHAR(255),
Z VARCHAR(255),
PRIMARY KEY (Y)
);
CREATE TABLE Table_3 (
Z VARCHAR(255),
W VARCHAR(255),
PRIMARY KEY (Z)
);
CREATE TABLE Table_4 (
X VARCHAR(255),
PRIMARY KEY (X)
);
CREATE TABLE Table_5 (
X VARCHAR(